# Cats vs. Dogs

Um sich ein initiales Bild über die *state-of-the-art* Trainingsmethoden für den `Cats vs. Dogs` Datensatz zu verschaffen wurde sich an diesem [Beispiel](https://www.kaggle.com/code/uysimty/keras-cnn-dog-or-cat-classification) orientiert.

**Matrikel-Nr.**: 1946566

**Requirements**:

- `tensorflow`
- `tensorflow-datasets`
- `keras` (zusätzlich zu tensorflow)
- `pyyaml`
- `h5py`
- `seaborn`

In [ ]:
import os
from pathlib import Path
import tensorflow as tf
import requests

# Disable TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Decide if you want to train a new model or load an existing one
LOAD_MODEL = True
# The default number of epochs to train the model is `20`
# For demonstration purposes and to safe time, I suggest to set it to `5`
EPOCHS = 3

In [ ]:
model_path = Path("./models")

if LOAD_MODEL and not Path(model_path / "cnn-0.88.h5").exists():
    req = requests.get("https://github.com/felixhoffmnn/machine-learning/raw/main/machine_learning/exam/models/cnn-0.88.h5")

    with open(model_path / "cnn-0.88.h5", "wb") as f:
        f.write(req.content)

## Data Analysis

---

Der `cats_vs_dogs` Datensatz besteht aus `23.262` Bildern von Katzen und Hunden. Die Bilder sind in 2 Klassen aufgeteilt: Katzen und Hunde (gelabelter Datensatz). Der Datensatz beinhaltet außerdem `1738` korrupte Bilder, die automatisch entfernt wurden.

`split=["train[:80%]", "train[80%:]"]` teilt den Datensatz in 2 Teile auf. Der erste Teil wird für das Training verwendet, der zweite Teil für die Validierung. Mittels `shuffle_files=True` werden die Bilder zufällig gemischt.

In [ ]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set("notebook", font_scale=1.5, style="white", rc={"figure.figsize":(20, 8)})

In [ ]:
(ds_train, ds_test), ds_info = tfds.load('CatsVsDogs', split=["train[:80%]", "train[80%:]"], shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
ds_examples = tfds.visualization.show_examples(ds_train, ds_info, rows=4, cols=4)

## Data Preparation

---

Wie sich aus der Recherche ergabt, gibt es verschiedene Möglichkeiten, die Bilder vorzubereiten. Im folgenden wenden wir drei Methoden an um das Bild zu normalisieren und zu skalieren. Außerdem werden die Bilder wenn nötig geshuffelt und in Batches aufgeteilt.

Mittels der Konvertierung aus dem `RGB` Farbraum in den `Grayscale` Farbraum wird die Anzahl der Farbkanäle von 3 auf 1 reduziert. Im Anschluss normalisieren wir das Histogramm der Bilder indem wir die Pixelwerte auf den Bereich von 0 bis 1 skalieren. Abschließend skalieren wir die Bilder auf `128 x 128` Pixel.

In [ ]:
def convert_to_gray(image, label):
    return tf.image.rgb_to_grayscale(image), label

def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label  # type: ignore

def resize_img(image, label):
    return tf.image.resize(image, (128, 128)), label

In [ ]:
def image_preprocessing(ds, batch_size=32, is_train=True):
    ds = ds.map(convert_to_gray, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(resize_img, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_train:
        ds = ds.shuffle(1000)
        
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

In [ ]:
ds_train_prep = image_preprocessing(ds_train)
ds_test_prep = image_preprocessing(ds_test, batch_size=64, is_train=False)

## Modeling and Evaluation

---

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import layers, models, optimizers, Sequential, regularizers
from keras.models import load_model
from tensorflow import keras
from datetime import datetime

checkpoint_path = Path("./checkpoints")

### Modeling

Aus der Recherche ergabt sich, dass eines der häufigsten Modelle für die Klassifikation ein **VGG 3** ist. Dieses besteht aus drei Convolution Layers, welche nacheinander versuchen die Bilder auf Basis von Merkmalen und Features zu klassifizieren. Um letztlich einen Binären Output zu erhalten, wird zunächst ein Flatten Layer verwendet, gefolgt von zwei Dense Layers. Der erste von beiden verarbeitet die Daten mit 512 Neuronen, der zweite mit 1 Neuron. 

Nachdem das aktuelle Bild mittels einem Convolution Layer verarbeitet wurde, wird das Ergebnis normalisiert und mittels einer MaxPooling Funktion auf die Hälfte reduziert. Dieser Vorgang wird 3 mal wiederholt. Das MaxPooling ist außerdem gefolgt von Dropout Layers, welche das Overfitting verhindern sollen.

**Iterationen**:

- Die erste Version des Modells beinhaltete ausschließlich Convolution Layers, MaxPooling, Flatten und Dense Layers &rarr; Das Modell hat stark *Overfitted*
- Das hinzufügen von Dropout Layers und Batch Normalization hat das Overfitting verhindert &rarr; Das Modell hat sich verbessert
- Durch das hinzufügen von `kernel_initializer` konnte die Trainingsgeschwindigkeit erhöht werden
- Padding vergrößert die `shape` &rarr; Das Modell hat sich verbessert

In [ ]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_uniform", padding="same", input_shape=(128, 128, 1)),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_uniform", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_uniform", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(512, activation="relu", kernel_initializer="he_uniform"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer=optimizers.RMSprop(), loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
time = datetime.now().strftime("%d_%m-%H_%M")

checkpoint_path = checkpoint_path / f"cats-dogs-{time}.ckpt"

Die folgenden **Callback-Funktionen** ermöglichen es das Training zu beenden, sobald sich die Genauigkeit nicht sonderlich verbessert. Außerdem verringern wir die `learning_rate` sobald die `validated_accuracy` nicht mehr steigt, sondern fällt. 

Durch die Verringerung der `learning_rate` konnten wir Genauigkeiten von bis zu `88` % erreichen.

In [ ]:
checkpoints = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, save_weights_only=True)
stop = EarlyStopping(patience=10, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001) # type: ignore

callbacks = [checkpoints, stop, lr_reduce]

**Warnung**: Das Training des Modells kann sehr lange dauern (ca. 1 Stunde). Daher wurde das Modell bereits trainiert und gespeichert. Somit kann das Modell direkt geladen werden.

In [ ]:
if not LOAD_MODEL:
    history = model.fit(ds_train_prep, validation_data=ds_test_prep, epochs=EPOCHS, callbacks=callbacks, use_multiprocessing=True)
    
    history_df = pd.DataFrame(history.history)
    print(history_df)
    
    highest = history_df["val_accuracy"].max().round(2)
    model.save(model_path / f"cats-dogs-{time}-{highest}.h5")

In [ ]:
if not LOAD_MODEL:
    fig, axs = plt.subplots(1, 2, figsize=(20, 7))

    sns.lineplot(x=history_df.index, y=history_df["loss"], ax=axs[0], label="loss")
    sns.lineplot(x=history_df.index, y=history_df["val_loss"], ax=axs[0], label="val_loss")
    sns.lineplot(x=history_df.index, y=history_df["accuracy"], ax=axs[1], label="accuracy")
    sns.lineplot(x=history_df.index, y=history_df["val_accuracy"], ax=axs[1], label="val_accuracy")

    axs[0].set_title("Loss")
    axs[1].set_title("Accuracy")

    axs[0].set_xlabel("Epochen")
    axs[0].set_ylabel("Loss")

    axs[1].set_xlabel("Epochen")
    axs[1].set_ylabel("Accuracy")

**Laden eines Modells**: Mittels des folgenden Codes kann ein bereits trainiertes Modell geladen werden. Zunächst muss `LOAD_MODEL` auf `True` gesetzt werden. Anschließend wird noch gefragt, welches Modell geladen werden soll.

Beim *Testen* des Ladens eines Modells wurde festgestellt, dass die Genauigkeit des Modells nicht identisch ist, wie als es trainiert wurde. Jedoch ist die Genauigkeit des Modells nach dem Laden noch immer sehr hoch.

In [ ]:
if LOAD_MODEL:
    models = [file for file in os.listdir(model_path) if file.endswith(".h5")]
    
    if len(models) > 0:
        for i, file in enumerate(models):
            print(f"{i}: {file}")
            
        model_index = int(input("\nSelect model: "))
        
        if model_index in range(len(models)):
            model = load_model(model_path / models[model_index])
            
            if model is not None:
                print(f"\nLoaded model: {models[model_index]}")
                history = model.evaluate(ds_test_prep, verbose=1, use_multiprocessing=True)
                
                print(history)

## Evaluation

In dieser Prüfungsleistung konnte ein `VGG3` mit einer Genauigkeit von `80 - 90` % trainiert werden. Als Datensatz wurde der **Cats vs. Dogs** Datensatz verwendet. Das Training wurde mittels eines `80:20` Split durchgeführt.

Um das Modell weiter zu verbessern könne man die Trainingsdaten verzerren, drehen und vergrößern. Dadurch könnte das Modell besser an Abweichungen bei den Testdaten angepasst werden.